In [8]:
from Laplace_JAXDense import solve_and_loss


In [1]:
# -*- coding: utf-8 -*-
'''
Created on Oct 2023

@authors:   Jamie Taylor (CUNEF)
            Manuela Bastidas (UPV/EHU)
            Tomas Teijeiro (BCAM)
            https://www.mathmode.science/
            
Updated on Jan 2025 by:
    Ángel Javier Omella
'''

# This code presents a simple implementation of Physics-Informed
# Neural Networks (PINNs) as a collocation method. -- <50 lines of PINNS--

# In this 1D example, we utilize Keras for constructing neural networks
# and JAX in the backend.


import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

import os

os.environ["KERAS_BACKEND"] = "jax"

import keras

In [11]:

# Set the random seed
np.random.seed(1234)
keras.utils.set_random_seed(1234)

dtype='float64' # double precision set to default in the SCR functions
jax.config.update("jax_enable_x64", True)
keras.backend.set_floatx(dtype)


# =============================================================================
#
#          Source code - PINNs H01 1D
#
# =============================================================================


## Define an approximate solution (u_nn): A neural network model
def make_model(neurons, n_layers, n_output, activation='tanh'):

    """
    Creates a neural network model to approximate the solution of
        int (grad u ).(grad v) - int f.v = 0

    Args:
        neurons (int): The number of neurons in each hidden layer.
        activation (str, optional): Activation function for hidden layers.

    Returns:
        keras.Model: A neural network model for the approximate solution.
    """

    xvals = keras.layers.Input(shape=(1,), name='x_input',dtype=dtype)

	# The input
    l1 = keras.layers.Dense(neurons, activation=activation, dtype=dtype)(xvals)

    ## ---------------
    #  The dense layers
    ## ---------------

    # First layer
    for l in range(n_layers-2):
        # Hidden layers
        l1 = keras.layers.Dense(neurons, activation=activation, dtype=dtype)(l1)
    # Last layer
    output = keras.layers.Dense(n_output, activation='softmax', dtype=dtype)(l1)

    model = keras.Model(inputs = xvals, outputs = output, name='model')

    # Print the information of the model u
    model.summary()

    return model

In [12]:
model = make_model(neurons=10, n_output=10, n_layers=1)

Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ x_input (InputLayer)            │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130 (1.02 KB)

 Trainable params: 130 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
output = model(jnp.array([1]))
output

Array([[0.08652977, 0.12354626, 0.09024292, 0.064689  , 0.13327708,
        0.09874309, 0.07757674, 0.16143284, 0.08189248, 0.08206984]],      dtype=float64)

In [15]:
output

Array([[0.08652977, 0.12354626, 0.09024292, 0.064689  , 0.13327708,
        0.09874309, 0.07757674, 0.16143284, 0.08189248, 0.08206984]],      dtype=float64)

In [16]:
jnp.cumsum(output)

Array([0.08652977, 0.21007602, 0.30031894, 0.36500794, 0.49828501,
       0.5970281 , 0.67460484, 0.83603768, 0.91793016, 1.        ],      dtype=float64)

In [46]:
class special_layer(keras.layers.Layer):
    def __init__(self, n_nodes, dimension, w_interior_initial_values = None, **kwargs):
        super().__init__(**kwargs)

        self.mobile_interior_vertices = self.add_weight(shape = (dimension, n_nodes), initializer = 'ones')
        
        if w_interior_initial_values is not None:
            self.mobile_interior_vertices.assign(w_interior_initial_values)
        else:
            self.mobile_interior_vertices.assign(jnp.ones((dimension, n_nodes)) / n_nodes)

    def call(self, inputs):
        return self.mobile_interior_vertices

def make_special_model(n_nodes, dimension, w_interior_initial_values):
    L = special_layer(n_nodes, dimension, w_interior_initial_values)
    xvals = keras.layers.Input(shape=(1,), name='x_input',dtype=dtype)
    output = L(xvals)
    model = keras.Model(inputs=xvals, outputs=output, name='model')
    return model

In [48]:
model = make_special_model(10, 1, None)

In [49]:
model(jnp.array([1]))

<Variable path=special_layer_19/variable_18, shape=(1, 10), dtype=float64, value=[[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]>